# Plot raw data

**Date:** July 2019

**Purpose:** create a plot of the data pr cycle with voltage on one y-axis and current on the other where the individual steps are within each span with added labels showing C-rate and values from the step_table

In [1]:
import os
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import bokeh
import cellpy
from cellpy.utils import plotutils
from cellpy.readers.core import xldate_as_datetime

In [2]:
%matplotlib inline
pd.options.display.max_columns = 100

In [3]:
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_cell = cellpy.get(filename)

(cellpy) - Making CellpyData class and setting prms
(cellpy) - Loading cellpy-file: ../testdata/hdf5/20160805_test001_45_cc.h5
(cellpy) - NumExpr defaulting to 4 threads.
(cellpy) - Created CellpyData object


### Checking individual cycles using plot_raw_cycle

In [4]:
my_cell.make_step_table()

In [5]:
data = my_cell.cell.raw
table = my_cell.cell.steps
summary = my_cell.cell.summary

In [6]:
table.head(3)

,cycle,step,sub_step,point_avr,point_std,point_min,point_max,point_first,point_last,point_delta,test_time_avr,test_time_std,test_time_min,test_time_max,test_time_first,test_time_last,test_time_delta,step_time_avr,step_time_std,step_time_min,step_time_max,step_time_first,step_time_last,step_time_delta,current_avr,current_std,current_min,current_max,current_first,current_last,current_delta,voltage_avr,voltage_std,voltage_min,voltage_max,voltage_first,voltage_last,voltage_delta,charge_avr,charge_std,charge_min,charge_max,charge_first,charge_last,charge_delta,discharge_avr,discharge_std,discharge_min,discharge_max,discharge_first,discharge_last,discharge_delta,ir_avr,ir_std,ir_min,ir_max,ir_first,ir_last,ir_delta,rate_avr,type,sub_type,info
0,1,1,1,144.5,83.282651,1,288,1,288,28700.000000,43351.128074,24985.406611,300.010482,86400.005396,300.010482,86400.005396,28698.995568,43351.128074,24985.406611,300.010482,86400.005397,300.010482,86400.005397,2.869900e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,2.994072,0.077039,2.873149,3.116708,3.097617,2.889160,-6.729599,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,ocvrlx_down,None,NaN
1,1,2,1,289.0,NaN,289,289,289,289,0.000000,86400.023870,NaN,86400.023870,86400.023870,86400.023870,86400.023870,0.000000,0.016058,NaN,0.016058,0.016058,0.016058,0.016058,0.000000e+00,6.628890e-07,NaN,6.628890e-07,6.628890e-07,6.628890e-07,6.628890e-07,0.00000,2.890700,NaN,2.890700,2.890700,2.890700,2.890700,0.000000,2.957000e-12,NaN,2.957000e-12,2.957000e-12,2.957000e-12,2.957000e-12,0.0,0.000000,NaN,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,56.486603,NaN,56.486603,56.486603,56.486603,56.486603,0.0,0.00,ir,None,NaN
2,1,3,1,662.0,215.207264,290,1034,290,1034,256.551724,96274.655793,13111.368851,86400.037738,127886.841385,86400.037738,127886.841385,48.017113,9874.634073,13111.368851,0.016017,41486.819665,0.016017,41486.819665,2.590092e+08,-1.522114e-04,4.651762e-07,-1.530085e-04,-1.484160e-04,-1.484160e-04,-1.526552e-04,-2.85631,0.767039,0.783011,0.049894,2.839894,2.839894,0.049894,-98.243109,2.959000e-12,0.0,2.959000e-12,2.959000e-12,2.959000e-12,2.959000e-12,0.0,0.000418,0.000555,3.145290e-10,0.001755,3.145290e-10,0.001755,5.580068e+08,56.486603,0.0,56.486603,56.486603,56.486603,56.486603,0.0,0.04,discharge,None,NaN


In [7]:
summary.head(3)

,Cycle_Index,Data_Point,Test_Time,DateTime,IR_Charge(Ohms),IR_Discharge(Ohms),End_Voltage_Charge(V),End_Voltage_Discharge(V),Charge_Capacity,Discharge_Capacity,Discharge_Capacity(mAh/g),Charge_Capacity(mAh/g),Cumulated_Discharge_Capacity(mAh/g),Cumulated_Charge_Capacity(mAh/g),Coulombic_Efficiency(percentage),Coulombic_Difference(mAh/g),Cumulated_Coulombic_Efficiency(percentage),Cumulated_Coulombic_Difference(mAh/g),Discharge_Capacity_Loss(mAh/g),Cumulated_Discharge_Capacity_Loss(mAh/g),Charge_Capacity_Loss(mAh/g),Cumulated_Charge_Capacity_Loss(mAh/g),Low_Level(percentage),High_Level(percentage),RIC(none),RIC_SEI(none),RIC_Disconnect(none),Charge_Endpoint_Slippage(mAh/g),Discharge_Endpoint_Slippage(mAh/g),Normalized_Cycle_Index
0,1,1457,168686.229691,2016-08-07 15:07:58.000000,48.372887,56.486603,1.000113,0.049894,0.001625,0.001755,1755.093529,1625.405999,1755.093529,1625.405999,92.610791,-129.687530,92.610791,-129.687530,NaN,NaN,NaN,NaN,0.892596,12.972323,NaN,NaN,NaN,129.687530,1884.781060,0.454151
1,2,2283,248276.169907,2016-08-08 13:14:25.000000,43.793484,14.831962,1.000113,0.049894,0.001700,0.001567,1567.475110,1699.563705,3322.568640,3324.969704,108.426838,132.088594,201.037629,2.401064,187.618419,187.618419,-74.157706,-74.157706,-0.016526,10.771885,0.034164,-0.035641,-0.045624,-2.401064,1565.074046,0.929022
2,3,3119,329048.542626,2016-08-09 11:40:42.000001,35.437023,14.545353,1.000113,0.049894,0.001732,0.001586,1585.720948,1731.507851,4908.289587,5056.477555,109.193730,145.786903,310.231359,148.187967,-18.245837,169.372582,-31.944146,-106.101852,-1.019929,9.894063,-0.062351,-0.102624,-0.064420,-148.187967,1437.532980,1.412819


In [8]:
data.head(3)

,Test_ID,Data_Point,Test_Time,Step_Time,DateTime,Step_Index,Cycle_Index,Is_FC_Data,Current,Voltage,Charge_Capacity,Discharge_Capacity,Charge_Energy,Discharge_Energy,dV/dt,Internal_Resistance,AC_Impedance,ACI_Phase_Angle
Data_Point,,,,,,,,,,,,,,,,,,
1,1,1,300.010482,300.010482,2016-08-05 16:21:22.999999,1,1,0,0.0,3.097617,0.0,0.0,0.0,0.0,-0.000123,0.0,0.0,0.0
2,1,2,600.014947,600.014947,2016-08-05 16:26:23.000000,1,1,0,0.0,3.100080,0.0,0.0,0.0,0.0,-0.000062,0.0,0.0,0.0
3,1,3,900.026371,900.026371,2016-08-05 16:31:23.000001,1,1,0,0.0,3.101312,0.0,0.0,0.0,0.0,-0.000123,0.0,0.0,0.0


In [9]:
my_cell.raw_limits

{'current_hard': 1e-13,
 'current_soft': 1e-05,
 'stable_current_hard': 2.0,
 'stable_current_soft': 4.0,
 'stable_voltage_hard': 2.0,
 'stable_voltage_soft': 4.0,
 'stable_charge_hard': 0.001,
 'stable_charge_soft': 5.0,
 'ir_change': 1e-05}

### Check using rawplot

In [10]:
plotutils.raw_plot(my_cell)

:Layout
   .Curve.Voltage_versus_time :Curve   [Test_Time_Hrs]   (Voltage)
   .Curve.I                   :Curve   [Test_Time_Hrs]   (Voltage)